[WOE](https://www.listendata.com/2015/03/weight-of-evidence-woe-and-information.html#Steps-of-Calculating-WOE)


In [1]:
#Load Required Packages
import pandas as pd
import numpy as np

ModuleNotFoundError: No module named 'pandas'